# Homework 1: File formats and TF binding sites

By the end of this homework you will...

* Know what common genome annotation formats are (BED, GTF)
* Used knowledge of UNIX commands to ask biological questions
* Know how to use the UCSC genome browser to download BED files
* Loaded `biotools` on TSCC to get access to the `bedtools` program
* Used `bedtools` to get promoters of genes, overlap them with transcription factor binding sites 
* Copied data from TSCC to your laptop
* Become more comfortable with reading and navigating documentation and man pages

## Log in to TSCC

Log in to tscc:

```
ssh username@tscc.sdsc.edu
```

## Clone the `biom262-2016` Github Repo

1. Read sections 10-12 from Software Carpentry's [Git intro](http://swcarpentry.github.io/git-novice/index.html).

You should have a `code` and `notebooks` directory already. If not, create them. Since the `biom262-2016` is essentially a bunch of code, we'll clone this into the `code` directory. Go into the `code` directory and clone the repo:

```
git clone https://github.com/biom262/biom262-2016.git
```

Now change directories to `weeks/week01/data`

* * *

## What is an annotation?
An annotation is a flag denoting a special characteristic to a particular base pair or string of base pairs in a DNA sequence.  The most common type of annotation assigns particular chromosomal coordinates to a gene ID (and its associated metadata.)  Genetic sequence is most commonly stored in strings of 'A', 'C', 'G', and 'T' (fasta format).  Annotations are sometimes bundled with sequence (gbk, genbank file) or downloaded separately in memory efficient formats like GTF and BED.

### BED Format: stuff in the genome

BED stands for "Browser Extensible Data" (informative I know .....) and is a standard format in bioinformatics for describing locations of stuff in the genome the basic format is described [here](https://genome.ucsc.edu/FAQ/FAQformat.html#format1), and here's a minimal example (from [`pybedtools`](https://github.com/daler/pybedtools/blob/master/pybedtools/test/data/a.bed) tests):

```
chr1	0	100	feature1	0	+
chr1	100	200	feature2	0	+
chr1	150	500	feature3	0	-
chr1	900	950	feature4	0	+
```

As you can see, the format is:

1. Chromosome name
2. Start of the feature (0-based, so the first base of the genome is "0")
3. Stop of the feature (0-based, so not inclusive). This is for computer science reasons, most programming langauges are written such that the "0th" item is the first thing. So then the "100th" item would then actually be the 101th element. To avoid the "off by one error" (a common problem in bioinformatics) then we use 0-based indexing, where `feature1` above starts at base 0 and ends at base 100, and thus is of length 100.
4. Name of the feature
5. Score of the feature, which is some integer from 0 to 1000. Some bed files come with a dot/period "`.`" here instead if it doesn't make sense for them to have a score, but there's programs that will complain that your `bed` file is improperly formatted so I end up using `awk` or something to fill that column with 1000 for every row.
6. Strand of the feature. If a strand doesn't make sense (e.g. for DNA methylation) then a dot/period "`.`"  is here.

### Get Transcription Factor Binding Site BED files from the UCSC Table Browser

Go to the [UCSC Genome Browser](http://genome.ucsc.edu/) and click "Tools > Table Browser", as shown below.

![](table_browser.png "UCSC Table Browser")

1. Go to the [Table Browser](http://genome.ucsc.edu/cgi-bin/hgTables) 
2. Choose "clade: Mammal," "genome: Human," and "assembly: Feb. 2009 (GRCh37/hg19)"
3. Choose "group: Regulation", "track: Txn Fac ChIP V2"
3. Use the default table (there are no others anyway)
4. Put "chr22" for the Region
5. Choose "output format: BED - browser extensible data". 
4. Save as "`tf.bed`"
4. Click "get output".
4. Do one bed record per gene (They really mean per "item", not a whole gene)
5. Click "get BED"

What does this file look like? Is it similar to what we had before? Let's use `head` to look at the first few lines, which by default is 10 lines.


Hint: Remember that the dollar sign ("`$`") indicates the shell command prompt that you're in, and shouldn't be included when you're copying commands.

```
$ head ~/Downloads/tf.bed     
chr22	16166497	16166741	CTCF	186	.	16166497	16166741	0	1	244,	0,
chr22	16201947	16202317	CTCF	603	.	16201947	16202317	0	1	370,	0,
chr22	16201988	16202252	YY1_(C-20)	63	.	16201988	16202252	0	1	264,	0,
chr22	16202021	16202231	Rad21	92	.	16202021	16202231	0	1	210,	0,
chr22	16202128	16202242	E2F6_(H-50)	269	.	16202128	16202242	0	1	114,	0,
chr22	16205233	16205683	SETDB1	356	.	16205233	16205683	0	1	450,	0,
chr22	16325696	16325900	NRSF	36	.	16325696	16325900	0	1	204,	0,
chr22	16560670	16560910	MafK_(ab50322)	102	.	16560670	16560910	0	1	240,	0,
chr22	16872258	16872628	CTCF	279	.	16872258	16872628	0	1	370,	0,
chr22	16872319	16872563	SMC3_(ab9263)	309	.	16872319	16872563	0	1	244,	0
```

### Copy `tf.bed` to TSCC

#### Mac/Linux

To copy data between computers, you can use "secure copy" or "`scp`". Just like with `cp` you do `cp filename newplace` you can do the same thing, but you have to specify the server name, and where on the server you want to put it. 

*Note: you are running this command from your **laptop**, because you need to be able to specify a unique address/URL for one of the file locations. Unless you have command line login and a static IP for your laptop (which takes a lot of work and is therefore unlikely), then you cannot run `scp` from TSCC to copy files to/from your laptop. You must always run `scp` from your laptop if you want to move files to/from a server.*

```
$ scp tf.bed username@tscc.sdsc.edu:~/code/biom262-2016/weeks/week01/data
```

Written another more generically, the `scp` command looks like

```
$ scp localfile username@server:/path/to/new/place/on/server
```

#### Windows

(Instructions compiled from [Cornell IT](http://www.it.cornell.edu/services/managed_servers/howto/file_transfer/fileputty.cfm) and [Analyzing Next-Gen Seq (ANGUS) data workshop](http://ged.msu.edu/angus/tutorials/using-putty-on-windows.html))

Install [PuTTY](http://www.chiark.greenend.org.uk/~sgtatham/putty/download.html). It doesn't need to be installed so you can put the file on your desktop or wherever you like to put your programs. You'll need both the `putty.exe` and `pscp.exe` files.

Open the windows command line 

![](http://www.it.cornell.edu/catc/cms/services/managed_servers/howto/file_transfer/images/filewindowsstart_1.jpg)
![](http://www.it.cornell.edu/catc/cms/services/managed_servers/howto/file_transfer/images/filewindowscmd_1.jpg)

At the command prompt, you can now enter same command as for Mac/Linux, but using `pscp` instead of `scp`.

```
c:\>pscp tf.bed username@tscc.sdsc.edu:~/code/biom262-2016/weeks/week01/data
```

Written another more generically, the `pscp` command looks like

```
c:\>pscp localfile username@server:/path/to/new/place/on/server
```

### Exercise 1: Get NFKB binding sites only

Use your UNIX skillz to filter the `tf.bed` file for only the NFKB TFs. Save this file as `tf.nkfb.bed`. This file should have 702 lines.

### Gencode gene annotations

We are interested in which gene's promoters these transcription factors bind to. We will use the [GENCODE](http://www.gencodegenes.org/) gene annotation, which is an aggregate of several gene annotation groups (ENSEMBL, HAVANA) to create a comprehensive set of human, and recently, mouse, gene annotations. We downloaded the annotations using [GENCODE v19](http://www.gencodegenes.org/releases/19.html), the last release using hg19, which as of writing in January 2016 has all the ENCODE transcription factor binding sites.

If you're interested in the difference between the GENCODE gene annotations and RefSeq genes, check out [this](http://www.biomedcentral.com/1471-2164/16/S8/S2) paper.

For this assignment, we have provided the gene annotations and sequence for human chromosome 22 (one of the smallest chromosomes), which are located in `biom262-2016/weeks/week01/data`

#### In case you are curious ...
This information is just so you know how grabbing just chromosome 22 for the different file formats was accomplished, here it is. It is not graded material. 

To get the `chr22` data from the gtf file, we can `grep` for lines that start with `chr22`, where the up-carat/power symbol "`^`" indicates the start of the line.

```
$ gunzip gencode.v24.basic.annotation.gtf.gz
$ grep '^chr22' gencode.v19.annotation.gtf > gencode.v19.annotation.chr22.gtf
```

To get the fasta file, we first had to index it using `samtools faidx`, and then we could use the same program to grab chromosome 22.

```
$ gunzip GRCh37.p13.genome.fa.gz 
$ samtools faidx GRCh37.p13.genome.fa.gz 
$ samtools faidx GRCh37.p13.genome.fa.gz chr22 > GRCh37.p13.chr22.fa 
```

#### Fasta and GTF formats

##### Fasta

The FASTA format is one of the oldest in bioinformatics and the simplest. The concept is this:

```
>sequence1_name
GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGCCCCCCCCCC
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGGGGGGGG
GGGGGGGGGG
>sequence2_name
CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCGGGGGGGGGG
GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGCCCCCCCCCC
CCCCCCCCCC
```

Where a sequence entry starts with a greater than sign "`>`" and any number of sequence lines can follow. The end of the sequence is indicated by the start of the next one (or the end of the file!). The example above shows two 110-base long sequences, with the first two rows of the sequence are exactly the same length (50bp), with a final hanging sequence (10bp). Some programs require that the lines be the same length, so if you're creating FASTA files by hand, it's recommended to use [BioPython's SeqIO](http://biopython.org/wiki/SeqIO) to generate them so they're correctly formatted.


We can use `head` to look at the first few lines (10 by default) of each of the files. The fasta file is kind of boring for the first few lines:


```
$ head GRCh37.p13.chr22.fa
>chr22
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
```

One question is, how many bases are in this file? Well, we'll need to ignore all rows of the file that start with "`>`", which we can do using `grep -v`, where "`-v`" *in**v**erts* the match and returns the non-matching lines. Here's the excerpt from `man grep`:

     -v, --invert-match
             Selected lines are those not matching any of the specified pat-
             terns.

To do this, we'll use `grep` to search for all lines that start with (startwith="`^`") the character "`>`". Remember that the greater than sign is a special character in unix that indicates to save the output to a file with that name, so we need to put quotes around the "regular expression" (aka search term) we are searching with to protect it from getting evaluated by the UNIX command line.

```
$ grep -v '^>' GRCh37.p13.chr22.fa | head
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
```

Let's make sure we've only removed the first line by looking at the line count.

```
wc -l 
 855078 GRCh37.p13.chr22.fa

```

And this command:

```
$ grep -v '^>' GRCh37.p13.chr22.fa | wc -l
  855077
```

So yes, we've removed exactly one line. Now we're ready to count the number of bases. Remember that the "`-l`" flag on "`wc`" indicated to count only the number of lines

```
$ grep -v '^>' GRCh37.p13.chr22.fa | wc   
  855077  855077 52159643
```

So that means there are 52,159,643 bases in chromosome 22. Cool!

##### GTF

Like the BED format, GTF encodes "stuff in the genome," but allows for hierarchy in the relationships between items and for additional annotations of the items. For example, GTFs can encode genes, which have transcripts as children, which have exons as children, which have coding sequences (CDS), untranslated regions (UTRs), stop codons, and more as children. This is not possible in BED files.

Read the GENCODE description of their [GTF](http://www.gencodegenes.org/data_format.html) format, which describes their columns and convenient anntations like whether the gene is protein-coding, the reading frame of coding sequences (CDS), and at which "level" a gene is annotated - verified, manually, or computationally.

Here is a screenshot of the GTF file from the GENCODE website.

![](gtf_example.png "GTF Example")

As you can see, it is *similar* but *different* than the bed file. Besides the columns, a key difference is that GTF files are 1-based but BED files are 0-based. Thus, if we have a feature which in BED-land:

```
chr1	0	100	feature1	0	+
```

Then in the GTF, we have:

```
chr1	source	gene	1	100	feature1	0	+	.
```

The GTF file is slightly easier to understand because the feature starts at 1 (and is 1-based so "1" is truly the first nucleotide) truly ends on base 99, and there is no more feature on base 100. How is this different for the negative strand?

For a bed file with the following 0-based coordinates, what would be the coordinates in the 1-based gtf file?

```
chr1	150	500	feature3	0	-
```

It'd be the same - the start is only changed. Why is that?

```
chr1	source	gene	151	500	feature3	0	-
```


Now let's look at GTF files on the command line. What if we do `head` on just the first two lines?

```
head -n 2 gencode.v19.annotation.chr22.gtf
chr22	HAVANA	gene	16062157	16063236	.	+	.	gene_id "ENSG00000233866.1"; transcript_id "ENSG00000233866.1"; gene_type "lincRNA"; gene_status "KNOWN"; gene_name "LA16c-4G1.3"; transcript_type "lincRNA"; transcript_status "KNOWN"; transcript_name "LA16c-4G1.3"; level 2; havana_gene "OTTHUMG00000140195.1";
chr22	HAVANA	transcript	16062157	16063236	.	+	.	gene_id "ENSG00000233866.1"; transcript_id "ENST00000424770.1"; gene_type "lincRNA"; gene_status "KNOWN"; gene_name "LA16c-4G1.3"; transcript_type "lincRNA"; transcript_status "KNOWN"; transcript_name "LA16c-4G1.3-001"; level 2; tag "basic"; havana_gene "OTTHUMG00000140195.1"; havana_transcript "OTTHUMT00000276574.1";
```


Oh that's annoying - GTF files are very wwwwwwwwwwwwwwiiiiiiiiiiiiiidddddddddddddddddddddeeeeeeeeeeeeeeeeeeeeeeeee and so the line wraps around and makes it are difficult to view in the terminal and line up columns. So we don't want to use `head`. Instead we'll use `less` with the `-S` flag. Check out `man less` to see **all** the options (there's a ton). Here's an excerpt from `man less` to show the function of `-S`:

       -S or --chop-long-lines
              Causes  lines  longer than the screen width to be chopped rather
              than folded.  That is, the portion of a long line that does  not
              fit  in  the  screen width is not shown.  The default is to fold
              long lines; that is, display the remainder on the next line.


```
$ less -S gencode.v19.annotation.chr22.gtf
```

It's nice because you can use the left and right arrow keys to navigate to different columns.

Here's a screenshot of the output:

![](gencode_v19_chr22_less-S.png "Example with 'less -S' to chop instead of wrap lines")

### Exercise 2: Extract only the genes from the GTF file

Use what you've learned about UNIX so far to get only the rows of `gencode.v19.annotation.chr22.gtf` that contain features of type "`transcript`", saving to a file called `gencode.v19.annotation.chr22.transcript.gtf`. Your file should contain `4459` lines.

## Use Bedtools to get promoters and overlap with TF binding sites

[Bedtools](http://bedtools.readthedocs.org/en/latest/) is a "swiss-army knife of tools for a wide-range of genomics analysis tasks". In particular, `bedtools` ***excels*** at "genome algebra", the adding and subtracting of genomic regions together. It is used ***EXTENSIVELY*** in bioinformatics and there are few projects that can live without it (or creating their own personal version but who wants to do that?)

### Load `biotools` on TSCC which includes Bedtools

First, try running `bedtools`. You should see this:

```
$ bedtools
bash: bedtools: command not found
```

This is because your shell doesn't know anything about the command `bedtools`. To load `bedtools` and other bioinformatics tools to your TSCC account, do

```
module load biotools
```

This command has no output. To make sure we have `bedtools` available, use `which` to see the full path to `bedtools`:

```
$ which bedtools
/opt/biotools/bedtools/bin/bedtools
```

(This is a TSCC-specific thing that the nice system administrators have set up for us but is not a general thing you can do on all servers. Maybe other nice sysadmins on other clusters do this but it is not guaranteed)

To see all available modules to load, do `module avail`:

```
[obotvinnik@tscc-login2 ~]$ module avail

-------------------------- /opt/modulefiles/applications/.gnu ---------------------------
atlas/3.10.2(default)     hdf5/1.8.14(default)      scalapack/2.0.2(default)
boost/1.55.0(default)     lapack/3.5.0(default)     slepc/3.5.3(default)
fftw/2.1.5                netcdf/3.6.2              sprng/2.0b(default)
fftw/3.3.4(default)       netcdf/4.3.2(default)     sundials/2.5.0(default)
gsl/1.16(default)         parmetis/4.0.3(default)   superlu/3.3(default)
hdf4/2.10(default)        petsc/3.5.2(default)      trilinos/11.12.1(default)

------------------------------- /opt/modulefiles/mpi/.gnu -------------------------------
mvapich2_ib/2.1rc2(default) openmpi_ib/1.8.4(default)

------------------------- /opt/modulefiles/applications/.intel --------------------------
atlas/3.10.2(default)     lapack/3.5.0(default)     scalapack/2.0.2(default)
boost/1.55.0(default)     mxml/2.9(default)         slepc/3.5.3(default)
fftw/2.1.5                netcdf/3.6.2              sprng/2.0b(default)
fftw/3.3.4(default)       netcdf/4.3.2(default)     sundials/2.5.0(default)
gsl/1.16(default)         papi/5.4.1(default)       superlu/3.3(default)
hdf4/2.10(default)        parmetis/4.0.3(default)   tau/2.23(default)
hdf5/1.8.14(default)      pdt/3.20(default)         trilinos/11.12.1(default)
ipm/2.0.3(default)        petsc/3.5.2(default)

------------------------------ /opt/modulefiles/mpi/.intel ------------------------------
mvapich2_ib/2.1rc2(default) openmpi_ib/1.8.4(default)

---------------------------- /usr/share/Modules/modulefiles -----------------------------
dot              module-info      null             rocks-openmpi_ib
module-git       modules          rocks-openmpi    use.own

----------------------------------- /etc/modulefiles ------------------------------------
openmpi-x86_64

------------------------------ /opt/modulefiles/compilers -------------------------------
cilk/5.4.6(default)           intel/2015.2.164
cmake/3.2.1(default)          mono/3.12.0(default)
gnu/4.9.2(default)            pgi/14.9(default)
guile/2.0.11(default)         python/1(default)
intel/2013_sp1.2.144(default) upc/2.20.0(default)

----------------------------- /opt/modulefiles/applications -----------------------------
abyss/1.5.2(default)        fsa/1.15.9(default)         mpi4py/1.3.1(default)
amber/14(default)           gamess/2014.12(default)     namd/2.10(default)
apbs/1.3(default)           gaussian/09.D.01(default)   namd/2.9
bbcp/14.09.02.00.0(default) globus/5.2.5                nwchem/6.5(default)
bbftp/3.2.1(default)        gmp/6.0.0a(default)         octave/3.8.2(default)
beagle/2.1(default)         gnutools/2.69(default)      polymake/2.13.1(default)
beast/1.8.0                 gromacs/5.0.4(default)      R/3.2.1(default)
beast/1.8.1(default)        idl/8.4(default)            rapidminer/6.1.0(default)
beast2/2.1.3(default)       jags/3.4.0(default)         scipy/2.7(default)
bioroll/6.2(default)        lammps/20141209(default)    siesta/3.2.5(default)
biotools/1(default)         matlab/2013a                stata/13.1(default)
blcr/0.8.5(default)         matlab/2013b                vasp/4.6
cp2k/2.5.1(default)         matlab/2014a                vasp/5.2.12
cpmd/3.17.1(default)        matlab/2014b(default)       vasp/5.2.12.gamma
cuda/6.5.19(default)        mkl/11.1.2.144(default)     vasp/5.3.5(default)
ddt/4.2.2(default)          mpc/1.0.3(default)          vtk/6.1.0(default)
eigen/3.2.3(default)        mpfr/3.1.2(default)         weka/3.7.12(default)
```

To see everything that's loaded, look in `/opt/biotools/`:

```
$ ls /opt/biotools/
bamtools   blat       cufflinks  GenomeAnalysisTK  miRDeep2  randfold    spades       trinity
bedtools   bowtie     dendropy   gmap_gsnap        miso      rseqc       squid        velvet
biopython  bowtie2    edena      htseq             picard    samtools    stacks       ViennaRNA
bismark    bwa        fastqc     idba-ud           plink     soapdenovo  tophat
blast      bx-python  fastx      matt              pysam     SOAPsnp     trimmomatic
```

### Exercise 2: Use `bedtools flank` to get the promoters

Get 2000bp upstream from each gene. The genome we are using is "`hg19`". Save this file as `gencode.v19.annotation.chr22.transcript.promoter.gtf`

Go to the documentation for [bedtools](http://bedtools.readthedocs.org/en/latest/) and find the entry for "`bedtools flank`".
`bedtools flank -h` is also informative (below)

```
$ bedtools flank -h

Tool:    bedtools flank (aka flankBed)
Version: v2.22.1
Summary: Creates flanking interval(s) for each BED/GFF/VCF feature.

Usage:   bedtools flank [OPTIONS] -i <bed/gff/vcf> -g <genome> [-b <int> or (-l and -r)]

Options: 
        -b      Create flanking interval(s) using -b base pairs in each direction.
                - (Integer) or (Float, e.g. 0.1) if used with -pct.

        -l      The number of base pairs that a flank should start from
                orig. start coordinate.
                - (Integer) or (Float, e.g. 0.1) if used with -pct.

        -r      The number of base pairs that a flank should end from
                orig. end coordinate.
                - (Integer) or (Float, e.g. 0.1) if used with -pct.

        -s      Define -l and -r based on strand.
                E.g. if used, -l 500 for a negative-stranded feature, 
                it will start the flank 500 bp downstream.  Default = false.

        -pct    Define -l and -r as a fraction of the feature's length.
                E.g. if used on a 1000bp feature, -l 0.50, 
                will add 500 bp "upstream".  Default = false.

        -header Print the header from the input file prior to results.

Notes: 
        (1)  Starts will be set to 0 if options would force it below 0.
        (2)  Ends will be set to the chromosome length if requested flank would
        force it above the max chrom length.
        (3)  In contrast to slop, which _extends_ intervals, bedtools flank
        creates new intervals from the regions just up- and down-stream
        of your existing intervals.
        (4)  The genome file should tab delimited and structured as follows:

        <chromName><TAB><chromSize>

        For example, Human (hg19):
        chr1    249250621
        chr2    243199373
        ...
        chr18_gl000207_random   4262

Tips: 
        One can use the UCSC Genome Browser's MySQL database to extract
        chromosome sizes. For example, H. sapiens:

        mysql --user=genome --host=genome-mysql.cse.ucsc.edu -A -e \
        "select chrom, size from hg19.chromInfo"  > hg19.genome
````


### Exercise 3: Use `bedtools intersect` to overlap TFs with 

Take a look at the diagrams in the documentation for `bedtools intersect` and think about how it can answer these questions:

* Given locations of genome methylation, which genes does it overlap?
* Given locations of RBP binding, which exons does it overlap?
* Given two CHIP-Seq experiments, which peaks are consistent between them?

Use `bedtools intersect` to find which promoters overlap with the NFKB binding sites. Call this file `gencode.v19.annotation.chr22.transcript.promoter.nfkb.gtf`.

#### Use `bedtools getfasta` to extract sequences

Read the documentation for [`bedtools getfasta`](http://bedtools.readthedocs.org/en/latest/content/tools/getfasta.html) and figure out how to  request the sequences in `fasta` format for the peaks which overlap in gene promoters

Save the file as `knownGene_exons.fasta`.



### Not required: HOMER

IF you want to use a program to get the binding motif of a transcription factor, you can do that with a `bed` file, the genome `fasta` file, and a motif finding program like [HOMER](http://homer.salk.edu/)